In [1]:
from libs import generation as gn
from libs import models as md
from libs import settings as st

from libs import utils as ut
from django.db import IntegrityError
import re
import numpy as np

import sqlalchemy as sa
from sqlalchemy import text
import pandas as pd
import datetime as dt
from sqlalchemy import inspect

path='data/rotten_tomatoes_top_movies.csv'

### Create table

In [2]:
inspector = inspect(st.conn_rotten_engine)
conn = st.conn_rotten_engine.connect()
if len(inspector.get_table_names()):
    md.Base.metadata.create_all(st.conn_rotten_engine)

### Data insertion

#### Genre base

In [3]:
df_genre = gn.dim_genre(path)
# ut.send_data(df_genre, st.conn_rotten_engine, 'dim_genre', 'dbo', True)

### Director base

In [4]:
df_director = gn.dim_director(path)
# ut.send_data(df_director, st.conn_rotten_engine, 'dim_director', 'dbo', True)

### Title base

In [5]:
df_title = gn.dim_title(path)
# ut.send_data(df_title, st.conn_rotten_engine, 'dim_title', 'dbo')

### Language base

In [6]:
df_language = gn.dim_language(path)
# ut.send_data(df_language, st.conn_rotten_engine, 'dim_language', 'dbo')

### Date base

In [7]:
start_date = ''.join(str(gn.generation_df_movies(path)['release_date_theaters'].min()).split(' ')[0])
end_date = ''.join(str(gn.generation_df_movies(path)['release_date_theaters'].max()).split(' ')[0])

df_date = gn.dim_date(start_date, end_date)

# ut.send_data(df_date, st.conn_rotten_engine, 'dim_date', 'dbo')

### Raking_Movies base

In [8]:
df_fact = gn.generation_df_movies(path, rel_columns=False)

df_fact = ut.create_foreing_key(df_fact, df_language, 'language','inner')
df_fact = ut.create_foreing_key(df_fact, df_title, 'title','inner')

df_fact.rename(columns={'release_date_theaters':"date_value"},inplace=True)
df_fact = pd.merge(df_fact, df_date, how='inner', on='date_value')
df_fact.drop(columns=['nk_date',	'year',	'month', 'day', 'date_value'],inplace=True)
df_fact.rename(columns={'id_x':'id','id_y':'release_date_theaters_id'},inplace=True)

df_fact.rename(columns={'release_date_streaming':"date_value"},inplace=True)
df_fact = pd.merge(df_fact, df_date, how='inner', on='date_value')
df_fact.drop(columns=['nk_date',	'year',	'month', 'day','date_value'],inplace=True)
df_fact.rename(columns={'id_x':'id','id_y':'release_date_streaming_id'},inplace=True)


#### Movies_Genre base

In [11]:
df_rel_movies_genre = gn.rel_movies_genre(df_genre, df_fact)
df_rel_movies_genre = df_rel_movies_genre[['genre_id','id']].reset_index(drop=True)
ut.send_data(df_rel_movies_genre, st.conn_rotten_engine, 'rel_movies_genre', 'dbo')

#### Movies_Director base

In [13]:
df_rel_movies_director = gn.rel_movies_director(df_director, df_fact)
df_rel_movies_director = df_rel_movies_director[['director_id','id']].reset_index(drop=True)
ut.send_data(df_rel_movies_director, st.conn_rotten_engine, 'rel_movies_director', 'dbo')

In [14]:
df_fact.drop(columns=['genre','director'],inplace=True)

ut.send_data(df_fact, st.conn_rotten_engine, 'fact_ranking_movie', 'dbo', True)